In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import datetime

In [2]:
df = pd.read_csv('/Users/josephbrennan/Desktop/Data/Merged/pson_merged.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df['Date'] = pd.to_datetime(df.Date)
df.index = pd.DatetimeIndex(df.Date)
df.drop('Date', axis=1, inplace=True)
df.head()

,Open,High,Low,Close,Volume,Pearson,education,Edexcel,publisher,penguin,books
Date,,,,,,,,,,,
2017-01-03,698.932507,704.052891,695.518917,696.372314,2289306,58,72,72,67,61,99
2017-01-04,697.225697,698.688411,684.424736,686.131531,3337718,71,76,83,89,65,100
2017-01-05,687.838354,695.945629,686.558257,693.385437,3200061,76,79,85,86,67,99
2017-01-06,692.958673,695.518866,689.545084,694.238770,1244707,66,70,74,71,73,92
2017-01-09,695.945545,701.065929,694.347144,698.505737,1714753,80,87,99,79,64,96


In [3]:
df.isna().sum()

Open         0
High         0
Low          0
Close        0
Volume       0
Pearson      0
education    0
Edexcel      0
publisher    0
penguin      0
books        0
dtype: int64

In [4]:
#Last row of data used to predict
compare_price = df.tail(1)
compare_price

,Open,High,Low,Close,Volume,Pearson,education,Edexcel,publisher,penguin,books
Date,,,,,,,,,,,
2021-06-29,834.400024,839.400024,823.400024,830.799988,1882235,83,91,60,87,80,94


In [5]:
#Get all data except last row (actual_price)
df = df.head(len(df)-1)
df

,Open,High,Low,Close,Volume,Pearson,education,Edexcel,publisher,penguin,books
Date,,,,,,,,,,,
2017-01-03,698.932507,704.052891,695.518917,696.372314,2289306,58,72,72,67,61,99
2017-01-04,697.225697,698.688411,684.424736,686.131531,3337718,71,76,83,89,65,100
2017-01-05,687.838354,695.945629,686.558257,693.385437,3200061,76,79,85,86,67,99
2017-01-06,692.958673,695.518866,689.545084,694.238770,1244707,66,70,74,71,73,92
2017-01-09,695.945545,701.065929,694.347144,698.505737,1714753,80,87,99,79,64,96
...,...,...,...,...,...,...,...,...,...,...,...
2021-06-22,855.000000,859.799988,850.200012,853.400024,942490,96,96,100,97,82,95
2021-06-23,853.400024,856.599976,844.400024,845.400024,931856,79,92,53,76,82,92
2021-06-24,849.599976,853.400024,832.200012,835.000000,1166885,79,100,72,96,79,93


In [6]:
#Save Feature Columns
feature_list = list(df.columns)
feature_list.remove('Close')
feature_list

['Open',
 'High',
 'Low',
 'Volume',
 'Pearson',
 'education',
 'Edexcel',
 'publisher',
 'penguin',
 'books']

In [7]:
df = pd.get_dummies(df)

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [9]:
#Features (X) and Labels (y)
y = df.iloc[:, 3].values
X = df.iloc[:, [0, 1, 2, 4, 5, 6, 7, 8, 9, 10]].values

In [10]:
#X = np.array(df)

In [11]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 100, shuffle=False)

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(793, 10)
(341, 10)
(793,)
(341,)


In [13]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [14]:
rf_random.best_params_

{'n_estimators': 2000,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': True}

In [15]:
#Model Evaluation Scores
from sklearn import metrics

def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_test, y_test)
base_accuracy = evaluate(base_model, X_test, y_test)

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Model Performance
Average Error: 3.0395 degrees.
Accuracy = 99.49%.
Model Performance
Average Error: 12.1799 degrees.
Accuracy = 97.61%.
Improvement of -1.89%.


In [16]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 1000, min_samples_split=2, min_samples_leaf=1, bootstrap=True, max_depth=100, max_features='auto')
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [17]:
from sklearn import metrics

print(metrics.r2_score(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.9720921038666716
Mean Absolute Error: 12.382351587200443
Mean Squared Error: 494.0754505463822
Root Mean Squared Error: 22.227808046372505


In [18]:
#Model Evaluation Scores
from sklearn import metrics

print('R-squared score: ', regressor.score(X_test, y_test))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
mape = np.mean(np.abs((y_test - y_pred) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

R-squared score:  0.9720921038666716
Mean Absolute Error: 12.382351587200443
Mean Squared Error: 494.0754505463822
Root Mean Squared Error: 22.227808046372505
Mean Absolute Percentage Error (MAPE): 2.43
Accuracy: 97.57


In [19]:
print(df.index.shape)
print(y_pred.shape)
print(y_train.shape)
predictions = np.concatenate((y_train, y_pred))
print(y_train[-1])
print(y_pred[-1])
print(predictions[-1])

(1134,)
(341,)
(793,)
546.73974609375
833.6268794555664
833.6268794555664
